In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import collections
import numpy as np
import sys

In [2]:

filename = 'tweets_corpus.txt'

DocID = {}
with open(filename) as fh:
    for line in fh:
        key, text = line.strip().split('\t', 1)
        DocID[key] = text.strip().split()
        DocID[key] = [word.lower() for word in DocID[key]]
        stop_words=list(stopwords.words('english'))   
        DocID[key] =[w for w in DocID[key] if not w in stop_words]


### Probelm 1: Inverted Index

In [3]:
inverted_index = dict()
for key in DocID:
    print (key)
    for text in DocID[key]:
        
        if text not in inverted_index:
            inverted_index[text]=[key]
        else:
            inverted_index[text].append(key)

81499877556760576
81500716438523904
81503002321616896
81507775422791680
81534165975171072
81535634019323904
81577509950459904
81582996083326976
81587643376336896
81600113016971264
81623945064890368
81644157432643584
81656304107651072
81673244926685184
81715158593966080
81716618236928000
81736742478155778
81842384404623360
81844590625304576
82461950231064576
82650970722533376
85032815321825280
85094773555335168
86441828815089664


In [4]:
inverted_index

{'bandaging': ['81499877556760576'],
 'paper-cuts': ['81499877556760576'],
 ',': ['81499877556760576',
  '81499877556760576',
  '81534165975171072',
  '81582996083326976',
  '81582996083326976',
  '81656304107651072',
  '81656304107651072',
  '81656304107651072',
  '81715158593966080',
  '81715158593966080',
  '81715158593966080',
  '81842384404623360',
  '82461950231064576',
  '82650970722533376',
  '82650970722533376',
  '85032815321825280',
  '85032815321825280',
  '85032815321825280',
  '85032815321825280',
  '85032815321825280',
  '85032815321825280',
  '86441828815089664',
  '86441828815089664',
  '86441828815089664'],
 'cheesecake': ['81499877556760576', '81600113016971264', '81716618236928000'],
 'dinner': ['81499877556760576'],
 'calling': ['81499877556760576'],
 'night': ['81499877556760576', '82650970722533376'],
 '.': ['81499877556760576',
  '81499877556760576',
  '81534165975171072',
  '81534165975171072',
  '81587643376336896',
  '81644157432643584',
  '81644157432643584'

In [5]:
inverted_index.keys()


dict_keys(['bandaging', 'paper-cuts', ',', 'cheesecake', 'dinner', 'calling', 'night', '.', "we're", 'doin', 'big', 'nyc', 'krispy', 'kremes', 'strawberry', 'trifles', 'since', 'started', 'gym', '*', 'cries', 'bacon/cheddar', 'slider', 'topped', 'w/fried', 'egg', '&', 'blue', 'cheese', 'w/avocado', 'purple', 'cherokee', 'tomato', 'nacho', 'w/', 'shirt', '!', 'uggghhh', 'aint', 'nuffin', 'piece', 'without', 'corners', '..', 'words', 'never', 'slice', 'bitch', 'tag_username', 'mmmm', '...', 'dreaming', 'squirrel', 'burger', '1st', "i'm", 'like', '1', 'year', 'younger', 'u', '2nd', 'age', 'number', '3rd', 'ima', 'cater', 'ur', 'wedding', 'wit', 'patty', 'n', 'rt', ':', 'want', 'steak', 'roll', 'right', 'think', 'imma', 'eat', 'befor', 'lay', 'havent', 'mixed', 'one', 'mostly', 'peach', 'little', 'white', 'cherry', 'dr', 'pepper', 'coke', 'etc', 'stomach', 'yelling', 'telling', 'get', 'ass', 'nd', 'somethin', 'went', 'got', 'cheesesticks', 'waterbottle', 'chocolate', 'mint', 'cookies', 'cr

In [6]:
print ("Dict key-value are : ") 
for i in inverted_index : 
    print(i, inverted_index[i]) 

Dict key-value are : 
bandaging ['81499877556760576']
paper-cuts ['81499877556760576']
, ['81499877556760576', '81499877556760576', '81534165975171072', '81582996083326976', '81582996083326976', '81656304107651072', '81656304107651072', '81656304107651072', '81715158593966080', '81715158593966080', '81715158593966080', '81842384404623360', '82461950231064576', '82650970722533376', '82650970722533376', '85032815321825280', '85032815321825280', '85032815321825280', '85032815321825280', '85032815321825280', '85032815321825280', '86441828815089664', '86441828815089664', '86441828815089664']
cheesecake ['81499877556760576', '81600113016971264', '81716618236928000']
dinner ['81499877556760576']
calling ['81499877556760576']
night ['81499877556760576', '82650970722533376']
. ['81499877556760576', '81499877556760576', '81534165975171072', '81534165975171072', '81587643376336896', '81644157432643584', '81644157432643584', '81673244926685184', '81715158593966080', '81715158593966080', '817151585

### Problem 2: Boolean Query

In [8]:
def Or(list1,list2,*args):
    
    ListA = list()
    for x in list1:
        ListA.append(x)
        for y in list2:
            if(x != y):
                ListA.append(y)
    
    return list(set(ListA))

In [9]:
def And(list1,list2,*args):
    
    ListA = list()
    for x in list1:
        for y in list2:
            if(x == y):
                ListA.append(x)       
    
    return ListA

In [10]:
# Egg and cheese
A = And(inverted_index['egg'],inverted_index['cheese'])
A

['81503002321616896', '81587643376336896', '82650970722533376']

In [11]:
# Chocolate and strawberry
B = And(inverted_index['chocolate'],inverted_index['strawberry'])
B

['81656304107651072', '81656304107651072', '85094773555335168']

In [12]:
# Chocolate or strawberry
C = Or(inverted_index['chocolate'],inverted_index['strawberry'])
C

['81716618236928000',
 '85094773555335168',
 '81715158593966080',
 '81842384404623360',
 '81500716438523904',
 '82461950231064576',
 '81656304107651072',
 '81623945064890368',
 '81844590625304576']

In [13]:
# (eggs and cheese ) OR (eggs and bacon)
D = Or(And(inverted_index['eggs'],inverted_index['bacon']),And(inverted_index['eggs'],inverted_index['cheese']))
D

['86441828815089664',
 '81673244926685184',
 '81736742478155778',
 '85032815321825280']

In [14]:
# eggs and cheese and bacon

E = And(inverted_index['eggs'],inverted_index['cheese'],inverted_index['bacon'])
E

['81673244926685184',
 '81736742478155778',
 '85032815321825280',
 '86441828815089664']

### Problem 3 : Tf-IDF score

In [25]:
def TF_IDF_term(term,d,DocID):
    total= len(inverted_index.values())
    TF = len(d)/total
    IDF = len(DocID)/len(d)
    
    return TF*IDF
    

In [28]:
def TF_IDF_query(q,doclist):
    query=q.lower()
    terms = []
    for w in query:
        if w.lower() != 'and' and w.lower() != 'or':
            terms.append(w)
    weights = dict()
    weight = 0
    for d in doclist:
        for term in terms:
            weight += TF_IDF_term(term,d,DocID)
        weights[d]= weight
    return weights

In [29]:
q = "eggs AND cheese"
doclist = A
TF_IDF_query(q,doclist)

{'81503002321616896': 1.8090452261306535,
 '81587643376336896': 3.618090452261308,
 '82650970722533376': 5.4271356783919575}

In [30]:
q = "Chocolate and strawberry"
doclist = B
TF_IDF_query(q,doclist)

{'81656304107651072': 5.788944723618087,
 '85094773555335168': 8.683417085427124}

In [31]:
q = "Chocolate or strawberry"
doclist = C
TF_IDF_query(q,doclist)

{'81716618236928000': 2.7738693467336692,
 '85094773555335168': 5.547738693467334,
 '81715158593966080': 8.321608040200994,
 '81842384404623360': 11.095477386934654,
 '81500716438523904': 13.869346733668314,
 '82461950231064576': 16.643216080401984,
 '81656304107651072': 19.417085427135685,
 '81623945064890368': 22.190954773869386,
 '81844590625304576': 24.964824120603087}

In [33]:
q = "(eggs and cheese ) OR (eggs and bacon)"
doclist = D
TF_IDF_query(q,doclist)

{'86441828815089664': 4.582914572864322,
 '81673244926685184': 9.16582914572863,
 '81736742478155778': 13.748743718592937,
 '85032815321825280': 18.33165829145728}

In [32]:
q = "eggs and cheese and bacon"
doclist = E
TF_IDF_query(q,doclist)

{'81673244926685184': 3.015075376884423,
 '81736742478155778': 6.03015075376884,
 '85032815321825280': 9.045226130653253,
 '86441828815089664': 12.060301507537666}